### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성청년SW·AI아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

### **Objectives**

1. 과제명: 토큰화/임베딩 심화 실습
2. 핵심 주제
    1) tokenizer를 이용하여 단어들을 토큰으로 변환하는 과정을 이해
    2) 토큰화된 토큰들을 임베딩 벡터로 변환하는 과정을 이해
    3) GPT 직접 구현해보기
3. 학습 목표
    1) 토크나이저가 무엇이고 토큰화가 무엇인지에 대해서 설명할 수 있다.
    2) 토큰화를 왜 하는지에 대해서 설명할 수 있다.
    3) 토큰화된 토큰들을 임베딩 벡터로 변환하는 과정을 이해할 수 있다.
    4) 임베딩 벡터를 이용하여 어떤 식으로 활용할 수 있는지 설명할 수 있다.
    5) GPT를 직접 구현할 수 있다.

4. 학습 개념
    1) 토큰화
    2) 임베딩 벡터
    3) GPT
  
5. 학습 방향
    - 실습은 아래 내용들을 깊게 파악하여 개념을 이해하며, 응용력을 갖추는 것이 목표입니다.
      - 토큰화
      - 임베딩
      - 디코더
    - 실습 코드는 조교가 직접 구현한 코드를 참고하며 학습합니다.
    - 자연스럽게 코드를 실행하면서 깊은 이해를 통해 개념을 이용하여 활용할 수 있을 정도의 깊이 있는 학습을 권장합니다.

6. 데이터셋 개요 및 저작권 정보
    - 데이터셋 명 : NSMC(Naver Sentiment Movie Corpus)
    - 데이터셋 개요 : 네이버 영화 감정분석 데이터셋
    - 데이터셋 저작권 : CC0 1.0
    - 데이터셋 명 : tinyshakespeare
    - 데이터셋 개요 : 셰익스피어의 작품을 텍스트로 변환한 데이터셋
    - 데이터셋 저작권 : MIT


### **Prerequisites**
```
numpy==2.0.2
pandas==2.2.2
tokenizers==0.21.4
transformers==4.55.2
plotly==6.2.0
nltk==3.9.1
sklearn==1.2.2
torch==2.8.0+cu126
```

- 아래 명령어를 복사해서 실행시켜주세요.

In [ ]:
%pip install numpy>=2.0.2 pandas>=2.2.2 tokenizers>=0.21.4 transformers>=4.55.2 plotly>=6.2.0 nltk>=3.9.1 scikit-learn>=1.2.2 torch==2.8.0+cu126 --index-url https://download.pytorch.org/whl

In [1]:
import torch
import torch.nn as nn
import numpy as np

# 시드 설정
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# 1. 토크나이저

- 학습 목표
  1. 토크나이저가 무엇이고 토큰화가 무엇인지에 대해서 설명할 수 있다.
  2. 토큰화를 왜 하는지에 대해서 설명할 수 있다.
- 학습 개념
  1. 토크나이저
  2. 토큰화
- 진행하는 실습 요약
  1. BPE 알고리즘을 사용하여 어떤 방식으로 토크나이저를 학습하는지 구현
  2. BPE 토크나이저를 직접 학습


## 1.1. Tokenizer

<blockquote>
<b>🧠 Tokenizer란?</b><br>
Tokenizer란 입력 텍스트를 <strong>토큰 단위 혹은 서브워드 단위</strong>로
쪼개주는 역할을 해주는 모듈입니다.
</blockquote>

`I'm a student of SSAFY!` 라는 입력 텍스트가 있다고 해봅시다.

  1. 단어(word) 단위로 쪼개면 `["I", "'", "m", "a", "student", "of", "SSAFY", "!"]`가 됩니다.
  2. 반면에 토큰(token) 단위로 쪼개면 `["I", "'", "m", "a", "student", "of", "SS", "##AF", "##Y", "!"]`가 됩니다.
  3. `student`같은 단어들은 토큰(token) 단위로 쪼개지지 않습니다.
  4. 반면에, `SSAFY`는 `SS, ##AF, ##Y` 처럼 토큰 단위로 쪼개졌습니다.
        - 여기서 `##`는 **같은 단어 안에서 나왔다**는 의미입니다.
        - `SS`는 한 단어의 **시작 토큰**, `##AF`과 `##Y`는 단어의 시작 토큰 이후에 단어를 완성하기 위해 필요한 토큰들입니다.
        - `["SS", "##AF", "##Y"]` 이후에 `.` 토큰이 나타났고, `##.`이 아니므로 현재 단어의 **끝 토큰**은 `##Y`입니다.
        <blockquote>
        <b>🤔 쪼개는 기준이 모호한거 같아요!</b><br>
        <b>토크나이저를 어떻게 만드냐(학습시키냐)</b>에 따라서 쪼개지는 방식도 다양하게 쪼개질 수 있습니다! <br>자세한 학습하는 방식에 대해서는 나중에 과제에서 실습해보겠습니다!</br>
        </blockquote>
  5. 이러한 토큰(token) 혹은 서브워드(sub-word) 단위는 단어(word)보다 작거나 글자(character)보다는 큰 단위로 쪼개는 것이 일반적입니다.
        - 반면에, `student`처럼 단어가 곧 토큰이 되는 경우도 존재합니다.
        - 이러한 차이점은 왜 발생하는 걸까요? - 이것은 토큰화를 하는 이유와도 연결됩니다.

<blockquote>
<b>📘 그렇다면 실제로 한번 진행해볼까요?</b><br>
sub-word 단위의 토크나이저는 BERT의 WordPiece Tokenizer를 사용하여 진행됩니다.
</blockquote>



In [2]:
# 단어 단위 토큰화

# 1. 단어 단위의 토크나이저를 불러옵니다. 여기서는 실습을 위해 간단하게 직접 구현하였습니다.
import re
def word_tokenize(text: str):
    lines = text.strip().split()
    tokens = []
    for line in lines:
        # 단어 또는 특수기호(.,!? 등)는 따로 분리
        line_tokens = re.findall(r"\w+|[^\w\s]", line)
        tokens.extend(line_tokens)

    return tokens

# 2. 토큰화를 진행합니다.
text = "I'm a student of SSAFY!"
tokens = word_tokenize(text)

# 3. 토큰화된 결과를 출력합니다.
print(tokens)

['I', "'", 'm', 'a', 'student', 'of', 'SSAFY', '!']


In [ ]:
from transformers import BertTokenizer

# 1. 학습이 완료가 된 토크나이저(WordPiece Tokenizer)를 불러옵니다.
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# 2. 토크나이저를 사용하여 토큰화를 진행합니다.
text = "I'm a student of SSAFY!"
tokens = tokenizer.tokenize(text)

# 3. 토큰화된 결과를 출력합니다.
print(tokens)

['I', "'", 'm', 'a', 'student', 'of', 'SS', '##AF', '##Y', '!']


## 1.2. 토큰화를 하는 이유
<blockquote>
<b>😲 왜 토큰화를 해야 할까요?</b><br>
가장 큰 이유는 Out of Vocabulary (OOV) 문제를 해결하기 위함입니다.
</blockquote>

토큰화를 하는 이유는 다음과 같은 이유들로 진행하게 됩니다.
1. Out of Vocabulary (OOV) 문제를 해결하기 위해
2. 단어의 접두사, 접미사 등 변형을 잘 반영하기 위해
3. 추론 속도 및 메모리 효율성 증가

이러한 이유들로 인해 토큰화를 하는 것이 중요합니다. 좀더 자세히 알아봅시다!

### 1.2.1. Out of Vocabulary (OOV) 문제

<blockquote>
<b>📘 Out of Vocabulary (OOV)</b><br>
OOV는 학습된 단어 사전에 없는 단어가 등장하는 문제를 의미합니다.
</blockquote>

예를 들어봅시다!
1. word 단위의 tokenizer에서 SSAFY라는 단어가 학습된 단어 사전에 없을 경우 SSAFY는 `[UNK]`라는 special token으로 변환됩니다.
    - special token은 일반적인 단어의 의미를 나타내는 것이 아닌 특정 상황에 대한 의미를 나타내기 위한 token입니다.
    - 예를 들어, `[CLS]`는 문장의 시작을 나타내는 token, `[SEP]`는 문장의 끝을 나타내는 token입니다.
    - `[UNK]`는 unknown token의 약자입니다. 즉, 학습되지 않은 단어가 등장하여 해당 의미를 모른다는 의미입니다.
    - SSAFY는 최근에 생긴 단어이거나 밈이거나 등등의 이유로 학습된 단어 사전에 없을 수 있습니다.
2. 반면에, subword 단위의 tokenizer에서 SSAFY라는 단어가 학습된 단어 사전에 없을 경우 SSAFY는 `SS, ##AF, ##Y`로 변환됩니다.
    - SSAFY는 `SS, ##AF, ##Y`로 변환된 이유는 SSAFY를 이루는 sub-word 단위로 변환함으로써 `[UNK]`이 아닌 정상적인 token으로 변환됩니다.

### 1.2.2. 단어의 접두사, 접미사 등 변형을 잘 반영하기 위해

<blockquote>
<b>📘 한국어 등의 교착어 특징을 가지는 경우</b><br>
교착어(agglutinative language)란 단어를 구성할 때 어근에 접두사(prefix), 접미사(suffix), 접중사(infix) 등을 차례대로 선형적으로 붙여 나가며, 각 형태소가 하나의 뜻이나 문법 범주(시제·격·수·인칭 등)를 담당하는 언어 유형을 의미합니다.
</blockquote>

예를 들어, 한국어의 경우 `먹다`, `먹고`, `먹으니` 등 `먹-`이라는 동사와 `다`, `고`, `니` 등의 접미사가 차례대로 붙여 나가는 것을 볼 수 있습니다.

이러한 특징을 잘 반영하기 위해 sub-word 단위의 tokenizer를 사용합니다.

### 1.2.3. 추론 속도 및 메모리 효율성 증가
<blockquote>
<b>📘 추론 속도 및 메모리 효율성 증가</b><br>
sub-word 단위의 tokenizer는 추론 속도 및 메모리 효율성을 <b>동시에</b> 개선할 수 있습니다.
</blockquote>

1. 글자(character) 단위의 tokenizer는 <b>너무 많은 token을 생성</b>하여 추론 속도가 떨어질 수 있습니다.
    - 예를 들어, `I'm a student of SSAFY!`를 변환하면 `I, 'm, a, s, t, u, d, e, n, t, o, f, S, S, A, F, Y, !`로 변환됩니다.
    - 이는 너무 많은 token을 생성하여 추론 속도가 떨어질 수 있습니다.
2. 반면에, 단어(word) 단위의 tokenizer는 <b>너무 많은 단어를 메모리에 저장</b>해야 하므로 메모리 효율성이 떨어질 수 있습니다.
    - 예를 들어, `I'm a student of SSAFY!`를 변환하면 `I, 'm, a, student, of, SSAFY, !`로 변환됩니다.
    - 글자(character) 단위보다는 추론 속도 측면에서 효율적입니다.
    - 하지만, 세상에 있는 모든 단어들(vocab table)을 메모리에 저장해야 하므로 메모리 효율성이 떨어질 수 있습니다.
3. 이러한 trade-off 관계를 적절히 해결하기 위해 sub-word 단위의 tokenizer를 사용합니다.
    - 글자 단위보다는 추론 속도 측면에서 효율적입니다.
    - 또한, word 단위보다 메모리에 올려야 할 token의 수가 적어 메모리 효율성이 높습니다.
    - 따라서, 추론 속도 및 메모리 효율성을 동시에 개선할 수 있는 sub-word 단위의 tokenizer를 사용합니다.

아래 코드를 실행하여 [UNK] 토큰이 생성되는지 확인해보세요.

In [4]:
# 토크나이저를 사용하여 토큰화를 진행합니다.
text = "놔뉸 쏴쁴 컄첌앺늬돠"
tokens = tokenizer.tokenize(text)

# 토큰화된 토큰들을 출력합니다.
print(tokens)

['[UNK]', '[UNK]', '[UNK]']


## 1.3. BPE 알고리즘
<blockquote>
<b>🧠 토크나이저는 어떤 원리로 학습될까요?</b><br>
Byte Pair Encoding (BPE) 알고리즘을 이용하여 빈도수 기반으로 학습합니다.
</blockquote>

초기에 텍스트를 압축하는 알고리즘으로 개발된 후, GPT 모델을 사전 학습할 때 토큰화를 위해 OpenAI에서 사용되었습니다. GPT, GPT-2, RoBERTa, BART 및 DeBERTa를 포함한 많은 트랜스포머 모델에서 사용됩니다.

간단한 BPE 알고리즘을 구현해보면서 토크나이저가 어떻게 학습되는지 이해해보겠습니다.

1. BPE 학습은 정규화 및 사전 토큰화 단계가 완료된 후, 말뭉치에 사용된 고유한 단어 집합을 계산하는 것으로 시작됩니다. 그런 다음 이러한 단어들을 구성하는데 사용된 모든 기호(글자)를 바탕으로 vocabulary를 구축합니다. 아주 간단한 예로서 말뭉치가 다음 네 단어를 사용한다고 가정해 봅시다:

- 학습할 단어들 : `low`, `lower`, `newest`, `widest`

- 기본 vocabulary는 ["l", "o", "w", "e", "r", "n", "t", "s", "i", "d"]가 됩니다.

2. BPE 알고리즘을 구현하기 위해서 우선 글자(토큰)쌍을 만듭니다. 여기서 "쌍"은 한 단어에서 두 개의 연속된 글자을 의미하고 글자는 처음에는 단일 문자입니다.

아래 코드를 완성하여 글자 쌍을 만드는 코드를 완성해주세요.

In [ ]:
def get_stats(corpus):
    """각 문자 쌍의 빈도를 계산"""
    pairs = {}
    for word, freq in corpus.items():
        # TODO: 띄어쓰기 및 계행을 기반으로 단어를 쪼개는 코드를 작성하세요.
        symbols = word.split()
        for i in range(len(symbols) - 1):
            # TODO: 글자 쌍을 생성하는 코드를 작성하세요. 글자쌍은 tuple로 저장됩니다.
            pair = (symbols[i], symbols[i + 1])
            if pair not in pairs:
                pairs[pair] = 1
            else:
                pairs[pair] += freq

    return pairs

3. 토크나이저 학습 과정에서 어떤 단계에서든 BPE 알고리즘은 가장 빈번하게 출현하는 글자 쌍을 검색합니다. 검색된 다빈도 글자 쌍이 병합되며 이러한 과정이 계속 반복됩니다.

아래 코드를 완성하여 BPE 알고리즘을 완성합니다.

In [ ]:
def merge_vocab(pair, corpus):
    """가장 빈도 높은 pair를 병합"""
    new_corpus = {}
    bigram = ' '.join(pair)
    replacement = ''.join(pair)
    for word in corpus:
        new_word = word.replace(bigram, replacement)
        new_corpus[new_word] = corpus[word]

    return new_corpus

def learn_bpe(corpus, num_merges):
    """BPE 알고리즘 학습"""
    for i in range(num_merges):
        pairs = get_stats(corpus)
        # pair들이 어떻게 구성되어 있는지 확인하고 싶다면 print(pairs)를 추가하여 그 과정을 확인해보세요!
        
        if not pairs:
            break
        
        # TODO: 가장 빈도 높은 pair를 선택하는 코드를 작성하세요.
        best = max(pairs, key=pairs.get)
        print(f"병합 {i+1}: {best} => {''.join(best)} ({pairs[best]}개)")
        corpus = merge_vocab(best, corpus)

    return corpus

BPE 알고리즘을 이용하여 실제 병합되는 과정을 확인해보겠습니다.

In [22]:
# 초기 corpus: 빈도 기반, 단어는 문자 사이에 공백 삽입
corpus = {
    'l o w': 5,
    'l o w e r': 2,
    'n e w e s t': 6,
    'w i d e s t': 3
}

# BPE 병합 실행
learned_vocab = learn_bpe(corpus, num_merges=5)

print("\nFinal Vocabulary:")
for word in learned_vocab:
    print(f"{word}: {learned_vocab[word]}")

병합 1: ('w', 'e') => we (7개)
병합 2: ('s', 't') => st (4개)
병합 3: ('l', 'o') => lo (3개)
병합 4: ('lo', 'w') => low (1개)
병합 5: ('lowe', 'r') => lower (1개)

Final Vocabulary:
low: 5
lower: 2
n e we st: 6
w i d e st: 3


이러한 방식으로 원하는 vocabulary 크기에 도달할 때까지 이 작업을 계속합니다.

## 1.4. Tokenizer 학습
<blockquote>
<b>📘 Tokenizer 학습</b><br>
실제 예제 코드로 Tokenizer 학습을 진행해보겠습니다.
</blockquote>

위에서 BPE 알고리즘을 통해 토크나이저가 빈도수 기반으로 글자들을 병합해나가면서 학습을 진행한다는 것을 확인할 수 있었습니다.

학습할 텍스트 데이터가 들어있는 파일을 준비합니다.

여기서는 NSMC(Naver Sentiment Movie Corpus) 데이터셋을 사용하겠습니다.

아래 명령어를 실행하여 데이터셋을 다운로드 받습니다.
(실습코드를 진행하셨다면 이미 데이터셋을 다운로드 받은 상태입니다.)

In [66]:
!wget https://github.com/e9t/nsmc/raw/master/ratings.txt

--2025-08-03 17:24:52--  https://github.com/e9t/nsmc/raw/master/ratings.txt
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [따라감]
--2025-08-03 17:24:52--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 19515078 (19M) [text/plain]
저장 위치: `ratings.txt'

ratings.txt         100%[===================>]  18.61M  11.1MB/s    /  1.7s    

2025-08-03 17:24:55 (11.1 MB/s) - `ratings.txt' 저장함 [19515078/19515078]



데이터셋을 확인해봅니다.

In [23]:
import pandas as pd
import os

file_list = os.listdir()
for file in file_list:
    if "ratings.txt" == file:
        print('학습에 필요한 파일이 존재합니다!', file)
        df = pd.read_table( (os.getcwd() + '/' + file), encoding='utf-8') # 데이터 프레임으로 보기 편하게 바꿔줍시다!
        df = df.dropna(how = 'any') # 널값을 없애줍니다!
        print('리뷰 갯수 :', len(df))
df.head()

학습에 필요한 파일이 존재합니다! ratings.txt
리뷰 갯수 : 199992


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


텍스트 데이터가 있는 'document'열만을 가져오고

해당 데이터를 txt 파일로 저장합니다.

In [24]:
with open((os.getcwd() + '/' + 'naver_review.txt'), 'w', encoding='utf8') as f:
    f.write('\n'.join(df['document']))

학습이 되어 있지 않은 빈 tokenizer 객체를 생성합니다.

여기서는 이해도를 돕기 위해 CharBPETokenizer를 불러옵니다. (OpenAI는 ByteLevelBPETokenizer입니다.)
CharBPETokenizer를 초기화할 때 파라미터들을 설정해주셔야 합니다.

#### 초기화 파라미터
- `suffix (default : </w>)`: `</w>`라는 문장의 마지막을 표시하는 special token을 추가합니다
- `split_on_whitespace_only (default: False)` : BPE는 '</w>'를 뜻하는 단어의 구별을 띄어쓰기와 마침표'.'을 기준으로 구별합니다. 이 설정을 True로 하면 마침표를 기준으로 '</w>'을 구별해주지 않습니다. 

#### 학습 파라미터
- `vocab_size (default: 30000)` : 단어사전 크기를 지정할 수 있습니다. 어떠한 값이 가장 좋다는 것은 없지만, 값이 클수록 많은 단어의 의미를 담을 수 있습니다.
- `min_frequency (default: 2)` : 최소 빈도수를 의미합니다. 만약 어떤 단어가 1번 나오면 vocab에 추가하지 않습니다.

In [41]:
from tokenizers import CharBPETokenizer

tokenizer = CharBPETokenizer(
    suffix='</w>',
    split_on_whitespace_only = True
)
tokenizer.train(
    files = 'naver_review.txt',
    vocab_size = 30000,
    min_frequency = 1,
)

vocab = tokenizer.get_vocab()
print(sorted(vocab, key=lambda x: vocab[x]))




['<unk>', '!', '"', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '★', '♡', '♥', 'ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅋ', 'ㅎ', 'ㅏ', 'ㅗ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', '가', '각', '간', '갈', '감', '갑', '값', '갓', '갔', '강', '갖', '같', '개', '객', '갠', '갱', '걍', '거', '건', '걸', '검', '겁', '것', '겉', '게', '겐', '겟', '겠', '겨', '격', '견', '결', '겹', '겼', '경', '계', '고', '곡', '곤', '골', '곱', '곳', '공', '과', '관', '광', '괜', '괴', '굉', '교', '구', '국', '군', '굳', '굴', '굿', '궁', '권', '귀', '규', '균', '그', '극', '근', '글', '금', '급', '기', '긴', '길', '김', '깊', '까', '깎', '깐', '깔', '깜', '깝', '깨', '꺼', '껄', '껏', '께', '껴', '꼈', '꼬', '꼭', '꼴', '꼽', '꽃', '꽝', '꽤', '꾸', '꾼', '꿀', '꿈

학습한 토크나이저를 기반으로 어떻게 토큰화를 하는지 확인합니다.

In [40]:
text = "I'm a student of SSAFY!"

encoded = tokenizer.encode(text)
print('🌱토큰화 결과 :',encoded.tokens)
print('🌱정수 인코딩 :',encoded.ids)
print('🌈디코딩 :',tokenizer.decode(encoded.ids))

🌱토큰화 결과 : ['I</w>', "'</w>", 'm</w>', 'a</w>', 'st', 'u', 'd', 'ent</w>', 'of</w>', 'S', 'SA', 'F', '<unk>', '!</w>']
🌱정수 인코딩 : [1572, 1844, 1344, 1091, 5714, 74, 58, 21118, 6559, 46, 28985, 36, 0, 1811]
🌈디코딩 : I ' m a student of SSAF!


## 2. 임베딩
- 학습 목표
  1. 토큰화된 토큰들을 워드 임베딩 벡터로 변환하는 과정을 이해할 수 있다.
- 학습 개념
  1. 워드 임베딩
- 진행하는 실습 요약
  1. nn.Embedding 레이어(혹은 간단한 dict lookup)를 사용하여 주어진 토큰 ID에 해당하는 임베딩 벡터를 조회하는 코드 
  2. 임베딩 시각화

<blockquote>
<b>🧠 임베딩이란?</b><br>
임베딩은 토큰화된 토큰들을 의미를 보존한 연속적인 벡터 공간으로 변환(매핑)하는 과정을 의미합니다.
</blockquote>

임베딩, 특히 워드 임베딩(word embeddings)은 단어(여기서는 토큰)를 의미를 보존한 연속적인 벡터 공간으로 변환하는 과정을 의미합니다.

좀더 구체적으로 말하면, 단어들을 밀집 벡터(dense vector)로 변환하는 과정을 의미합니다.

### 2.1. 밀집 벡터(dense vector)

밀집 벡터는 모든 차원을 연속적인 실수값으로 표현한 벡터입니다.

밀집 벡터와 반대되는 희소 벡터(Sparse Vector)와 비교해보면 다음과 같습니다.

<table>
  <thead>
    <tr>
      <th>개념</th>
      <th>설명</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Dense vector</td>
      <td>모든 차원에 실수값이 골고루 존재 ([0.23, -1.5, 0.88, 0.01, ...])</td>
    </tr>
    <tr>
      <td>Sparse vector</td>
      <td>대부분이 0이고, 일부 위치에만 값이 있음 (예: one-hot [0, 0, 1, 0, 0, ...])</td>
    </tr>
  </tbody>
</table>


<blockquote>
<b>🧠 밀집 벡터를 사용하는 이유는 희소 벡터보다 더 적은 차원으로 표현할 수 있기 때문입니다.</b><br>
즉, 밀집 벡터는 희소 벡터보다 더 많은 정보를 적은 차원에 담는 벡터라고 볼 수 있습니다.
</blockquote>

그렇다면 워드 임베딩으로 어떻게 정보를 담게 되는지 살펴보겠습니다.

### 2.2. 워드 임베딩

워드 임베딩이란 단어들을 벡터화하여 표현하는 방법입니다.
사전에 학습된 워드 임베딩을 통해 워드 임베딩의 차원이 무엇을 의미하는지를 파악해봅시다!

아래 코드는 다양한 의미로 그룹을 지을 수 있는 단어들을 나열했습니다.

- Age group : "infant", "child", "girl", "boy"
- Adult group : "man", "woman", "grandfather"
- Royal group : "prince", "princess", "king", "queen", "monarch"

위의 3가지 의미로 각 단어들의 의미를 묶어볼 수 있습니다. 각 단어들을 그룹으로 묶었다는 것은 아래와 같은 의미를 가집니다.

<blockquote>
<b>🧠 그룹 내의 단어들은 유사한 단어들이라고 표현할 수 있습니다.</b><br>
각 그룹을 기준으로 유사한 단어들을 묶어내고 그 그룹들이 실제로 임베팅 공간에 어떻게 표현되는지 확인해봅니다.
</blockquote>

특히 임베딩 공간에서 유사한 단어들은 어디에 위치하는지를 유심하게 확인해보세요.

In [85]:
words = [
    "infant", "child", "girl", "boy",       # 👶 Age group
    "man", "woman", "grandfather",          # 🧑 Adult group
    "prince", "princess", "king", "queen", "monarch"  # 👑 Royal group
]

<blockquote>
<b>🧠 먼저 학습이 완료된 word embeddings의 경우 어떻게 나오는지 확인해봅시다.</b><br>
이미 잘 학습된(사전학습된) word embeddings을 불러와서 사용해보겠습니다.
</blockquote>

In [84]:
from transformers import BertModel, BertTokenizer

# 사전학습된 BERT 토크나이저 및 모델 불러오기
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

print("토크나이저의 vocab size : ", tokenizer.vocab_size)

# word embedding 꺼내기
word_embeddings = model.embeddings.word_embeddings
print("워드 임베딩의 차원 : ", word_embeddings.weight.shape)

토크나이저의 vocab size :  30522
워드 임베딩의 차원 :  torch.Size([30522, 768])


코드를 실행해본 결과 `word_embeddings`의 차원은 `30522 x 768` 입니다.
- 행인 `30522`는 vocab의 단어 수를 의미합니다.
    - 이전에 저희는 토크나이저를 학습해보면서 `vocab_size`를 30000으로 설정하고 학습을 했습니다.
    - 이번에 불러온 토크나이저는 `vocab_size=30522`인 토크나이저를 의미합니다.
    - 그리고 word_embeddings의 행은 vocab_size와 동일합니다.
- 열인 `768`은 밀집 벡터에서 표현하는 차원 값을 의미합니다.
    - 단어의 의미를 표현하기 위한 값는 마음대로 설정해도 됩니다.
    - 여기서는 256 * 3 = 768의 값으로 설정했습니다.
    - 보통 768, 1024, 2048 등 2의 제곱수로 설정됩니다.

<blockquote>
<b>🧠 즉, word_embeddings의 차원은 </b><br>
모델이 표현 가능한 단어(토큰)의 수 x 밀집 벡터의 차원입니다.
</blockquote>

차원은 정확히 어떤 값이 좋다는 값은 없습니다. 하지만 아래와 같은 특징이 있습니다.

|  | 차원이 클 때 (embedding_dim ↑) | 차원이 작을 때 (embedding_dim ↓) |
| --- | --- | --- |
| **표현력** | 더 정교하고 미묘한 의미를 표현 가능 | 표현력 부족 가능성 있음 |
| **계산 속도** | 계산이 느리고 메모리 사용량이 많음 | 계산이 빠르고 메모리 효율적 |
| **과적합** | 과적합 가능성 증가 | 과적합 가능성 감소 |


그러면 이제 실제 단어들이 임베딩 공간 상에서 어떻게 표현되는지 확인해봅시다.

아래 코드를 실행하여 단어들을 token id로 매핑한 후 매핑한 토큰들을 임베딩 공간 상에서 표현하는 방법을 확인해보세요.

In [ ]:
words_to_ids = {word: tokenizer.convert_tokens_to_ids(word) for i, word in enumerate(words)}
print("입력으로 들어가는 토큰들과 그에 해당하는 id 값들 : ", words_to_ids)

input_tensor = torch.tensor([words_to_ids[word] for word in words])
print("입력으로 들어가는 토큰들의 id 값들 : ", input_tensor)
print("입력으로 들어가는 텐서의 차원 : ", input_tensor.shape)
embeddings_vector = word_embeddings(input_tensor)
print("임베딩 벡터의 차원 : ", embeddings_vector.shape)

입력으로 들어가는 토큰들과 그에 해당하는 id 값들 :  {'infant': 10527, 'child': 2775, 'girl': 2611, 'boy': 2879, 'man': 2158, 'woman': 2450, 'grandfather': 5615, 'prince': 3159, 'princess': 4615, 'king': 2332, 'queen': 3035, 'monarch': 11590}
입력으로 들어가는 토큰들의 id 값들 :  tensor([10527,  2775,  2611,  2879,  2158,  2450,  5615,  3159,  4615,  2332,
         3035, 11590])
입력으로 들어가는 텐서의 차원 :  torch.Size([12])
임베딩 벡터의 차원 :  torch.Size([12, 768])


결과를 보시면 임베딩 벡터의 차원이 `30522 X 768`에서 `12 X 768`으로 줄어듭니다.
이는 각 입력 토큰에 해당하는 `id` 값이 인덱스가 되어 해당 인덱스 행의 임베딩 벡터를 가져오게 됩니다.

예를 들어, `10527` id를 가지는 토큰 `infant`은 `10527`번째 행의 임베딩 벡터를 가져오게 됩니다.

### 2.3. 임베딩 시각화

그러면 임베딩을 시각화해보겠습니다.

아래 코드는 임베딩을 시각화하기 위한 코드입니다.
코드에 대한 분석을 진행하지 않아도 좋습니다!

In [ ]:
def visualize_embeddings(embeddings_vector, words, word_colors):
    import plotly.graph_objects as go
    from sklearn.decomposition import PCA
    
    """
        단어 임베딩을 3차원 구의 형태로 시각화
        이 코드는 굳이 이해할 필요가 없습니다. 
    """
    
    if isinstance(embeddings_vector, torch.Tensor):
        embeddings_vector = embeddings_vector.detach().numpy()
    
    
    # 3차원 구의 형태로 표현하기 위해 PCA로 차원축소
    reduced = PCA(n_components=3).fit_transform(embeddings_vector)
    normed = reduced / np.linalg.norm(reduced, axis=1, keepdims=True)

    x, y, z = normed[:, 0], normed[:, 1], normed[:, 2]

    # 구 표면 좌표 생성
    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x_sphere = np.outer(np.cos(u), np.sin(v))
    y_sphere = np.outer(np.sin(u), np.sin(v))
    z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

    # 구형 표면
    sphere = go.Surface(
        x=x_sphere, y=y_sphere, z=z_sphere,
        colorscale='Greys', opacity=0.1, showscale=False
    )

    # 점 + 텍스트 (색상 반영)
    points = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers+text',
        marker=dict(size=6, color=word_colors),
        text=words,
        textposition='top center'
    )

    # 원점 → 단어 벡터 연결 선
    lines = [
        go.Scatter3d(
            x=[0, x[i]], y=[0, y[i]], z=[0, z[i]],
            mode='lines',
            line=dict(width=2, color=word_colors[i]),
            showlegend=False
        )
        for i in range(len(words))
    ]

    # 레이아웃
    layout = go.Layout(
        title="3D Word Embeddings with Semantic Coloring",
        margin=dict(l=0, r=0, t=40, b=0),
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            aspectmode='cube'
        )
    )

    fig = go.Figure(data=[sphere, points] + lines, layout=layout)
    fig.show()

이제 아래 코드를 실행하여 임베딩 공간상에 토큰들을 위치시켜보고 그 의미를 파악해봅시다!

In [89]:
# 단어별 그룹 매핑
group_map = {
    "age": ["infant", "child", "girl", "boy"],
    "adult": ["man", "woman", "grandfather"],
    "royalty": ["prince", "princess", "king", "queen", "monarch"]
}

# 각 단어에 그룹별 색상 지정
color_map = {
    "age": "red",
    "adult": "blue",
    "royalty": "green"
}

word_colors = []
for w in words:
    for group, word_list in group_map.items():
        if w in word_list:
            word_colors.append(color_map[group])
            break

visualize_embeddings(embeddings_vector, words, word_colors)

위의 임베딩 공간을 확인해보시면 각 색깔들이 그룹을 의미합니다.

각 그룹들의 단어들이 비슷한 위치에 있는 것을 확인하실 수 있습니다.

이러한 임베딩 공간을 확인하는 것이 워드 임베딩의 핵심적인 개념입니다.

<blockquote>
<b>🧠 좀더 정확히 표현하자면, 유사한 단어일수록 방향이 비슷하고 벡터 간 각도가 작다는 것을 의미합니다. </b><br>
유사도를 측정하는 방식은 코사인 유사도(cosine similarity)를 사용합니다. 코사인 유사도에서의 핵심은 벡터 간 각도를 측정하는 것입니다.
</blockquote>

우리 눈에 보이는 것은 가까운 거리상에 존재한다고 보일 수 있습니다.

하지만, 벡터 상에서는 거리보다는 방향과 각도가 중요하다는 것을 꼭 기억하시기 바랍니다.

위에서의 예시는 이미 잘 학습이 되어 있는 워드 임베딩을 확인하였습니다.

반면에 학습이 되어있지 않은 워드 임베딩을 확인해보겠습니다.

`nn.Embedding()` 매서드를 통해 임베딩을 초기화할 수 있습니다.
또한, 학습된 임베딩이 아니므로 `vocab_size` 또한 저희가 정의한 12개의 단어만 있다고 가정해보겠습니다.

In [ ]:
# TODO: embeddings layer를 구현하세요.
embedding_layer = nn.Embedding(len(words), 768)

In [ ]:
words_to_ids = {word: i for i, word in enumerate(words)}
print("입력으로 들어가는 토큰들과 그에 해당하는 id 값들 : ", words_to_ids)

input_tensor = torch.tensor([words_to_ids[word] for word in words])
print("입력으로 들어가는 토큰들의 id 값들 : ", input_tensor)
print("입력으로 들어가는 텐서의 차원 : ", input_tensor.shape)
embeddings_vector = embedding_layer(input_tensor)
print("임베딩 벡터의 차원 : ", embeddings_vector.shape)

입력으로 들어가는 토큰들과 그에 해당하는 id 값들 :  {'infant': 0, 'child': 1, 'girl': 2, 'boy': 3, 'man': 4, 'woman': 5, 'grandfather': 6, 'prince': 7, 'princess': 8, 'king': 9, 'queen': 10, 'monarch': 11}
입력으로 들어가는 토큰들의 id 값들 :  tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
입력으로 들어가는 텐서의 차원 :  torch.Size([12])
임베딩 벡터의 차원 :  torch.Size([12, 768])


결과를 보시면 이전에 했던 것과 동일하게 `12 X 768`의 임베딩 벡터를 얻을 수 있습니다.

이제 이 임베딩 벡터를 시각화해보겠습니다.

In [96]:
# 단어별 그룹 매핑
group_map = {
    "age": ["infant", "child", "girl", "boy"],
    "adult": ["man", "woman", "grandfather"],
    "royalty": ["prince", "princess", "king", "queen", "monarch"]
}

# 각 단어에 그룹별 색상 지정
color_map = {
    "age": "red",
    "adult": "blue",
    "royalty": "green"
}

word_colors = []
for w in words:
    for group, word_list in group_map.items():
        if w in word_list:
            word_colors.append(color_map[group])
            break

visualize_embeddings(embeddings_vector, words, word_colors)

잘 학습된 워드 임베딩과 다르게 학습되지 않은 워드 임베딩은 상대적으로 그룹끼리 모여있지 않은 것을 확인하실 수 있습니다.

이처럼 학습되지 않은 워드 임베딩으로는 그 단어의 의미를 명확하게 표현할 수 없습니다. 

또한 **잘 학습된 워드 임베딩은 각 단어들간의 관계(유사한 정도)를 명확하게 표현**할 수 있습니다.

이전 질문으로 돌아가보자면,

<blockquote>
<b>🧠 워드 임베딩으로 어떻게 정보를 담게 될까요? </b><br>
각 단어들을 벡터 상의 좌표로 표현하고 그 좌표는 방향과 각도를 가지게 되며, 방향과 각도로 단어들간의 관계 정보를 담게 됩니다.
</blockquote>

# 3. GPT 구현
- 학습 목표
  1. GPT를 구현하여 어텐션이 어떻게 사용되는지 파악합니다.
- 학습 개념
  1. GPT
- 진행하는 실습 요약
  1. GPT 사전학습을 실제로 진행해봅니다.

간단한 GPT를 직접 구현해보면서 모델 아키텍쳐를 이해해보겠습니다.

최근에 나오는 OpenAI의 GPT, Google의 Gemini, Meta의 Llama 등은 모두 이러한 GPT의 아키텍쳐를 기반으로 다양하게 변형하거나 학습들 다양하게 한 모델들입니다.

즉, 이번 GPT 구현 실습을 해본다면 모델에 대한 깊은 이해도를 가져가실 수 있습니다.

모델에 대한 아키텍쳐를 어느정도 이해한 다음애 모델의 특성을 파악한다면 더더욱 이해도가 높아질 것입니다.

config를 설정합니다.

In [25]:
from dataclasses import dataclass

@dataclass
class Config:
    # The size of hidden state in transformer, also called d_model
    hidden_size: int = 512
    # The size of hidden state in MLP in the decoder block
    ff_hidden_size: int = 4 * 512
    # The number of decoder blocks
    num_hidden_layers: int = 2
    # Dropout rates for all modules that need dropout
    dropout_rate: float = 0.1
    vocab_size: int = 10000
    max_seq_len: int = 128
    num_heads: int = 1

config = Config()

GPT-2는 아래와 같은 구조를 가지고 있습니다.

![image](https://raw.githubusercontent.com/Ssunbell/TIL/refs/heads/master/assets/GPT-2.png)

오른쪽 attention Head에서부터 왼쪽 GPT2으로 차례대로 구현하도록 하겠습니다.

### Attention Head

![image](https://raw.githubusercontent.com/Ssunbell/TIL/refs/heads/master/assets/MultiHeadAttention.png)


GPT-2 스타일의 compact multi-head causal self-attention 블록:
- 입출력 형태: 입력과 출력은 모두 같은 크기 $(B, T, H)$입니다. (여기서 $B$는 배치 크기, $T$는 시퀀스 길이, $H$는 hidden dimension)
- 헤드 분할: $n_h$개의 attention head로 쪼갠 뒤, 각 head 차원은 $d = H/n_h$가 됩니다.
- 구성 요소:
	1.	하나의 선형층에서 Q/K/V를 동시에 생성
	2.	Q와 K로 dot-product를 계산하고 크기를 $d$로 나눈 뒤 softmax (attention score 계산)
	3.	미래 토큰을 못 보게 causal mask 적용
	4.	softmax 후 dropout으로 정규화
	5.	attention score로 V를 가중합하여 context 생성
	6.	여러 head를 다시 합쳐 concat
	7.	최종적으로 projection과 dropout으로 출력

즉, “현재 토큰이 과거 정보 중 어디를 집중해서 볼지(attend할지)“를 계산하고, 그 가중치를 이용해 Value를 섞어주는 역할을 합니다.

표기법 & 텐서 크기
- $X \in \mathbb{R}^{B\times T\times H}$: 입력 시퀀스 임베딩
- 이를 $n_h$개의 head로 나누면 각 head는 $d = H/n_h$ 차원을 가짐
- head 단위로 보면 $Q,K,V \in \mathbb{R}^{B\times n_h\times T\times d}$

즉, 입력이 여러 개의 작은 “주의 집중 장치(head)”로 나뉘어 병렬 계산됩니다.

1) Q/K/V 투영

```python
self.c_attn = nn.Linear(H, 3H, bias=False)
qkv = self.c_attn(x)
q, k, v = qkv.split(H, dim=-1)
```

입력을 바로 Q/K/V로 만드는 것이 아니라,
하나의 큰 선형층이 입력을 받아 세 개의 행렬로 동시에 변환합니다.
- 수식:
$$
[Q’, K’, V’] = X W_{qkv}
$$
- 이것은 곧 $Q’,K’,V’$가 각각 $X$에 대해 다른 projection($W_Q,W_K,W_V$)을 적용한 결과라는 뜻입니다.

이후 head 단위로 reshape 해주어 $Q,K,V$를 $(B, n_h, T, d)$ 크기로 변환합니다.
→ 이렇게 하면 “각 head가 자기만의 관점으로 입력을 본다”고 해석할 수 있습니다.

2) Scaled dot-product attention score

```python
attn_scores = (q @ k.transpose(-2, -1)) / sqrt(head_dim)
```

각 head에서, Query와 Key의 내적을 구해 “서로 얼마나 유사한지” 측정합니다.
- 수식:
$$
S_h = \frac{Q_h K_h^\top}{\sqrt{d}}
$$

Query는 “내가 지금 어떤 정보가 필요하다”라는 질문이고,

Key는 “내가 가진 정보의 특징”이라고 생각할 수 있습니다.

둘의 내적이 크면, 그 Key에 해당하는 Value를 더 많이 참고하게 됩니다.

$\sqrt{d}$로 나눠주는 이유는 차원이 커질수록 내적 값이 커져 softmax가 불안정해지기 때문입니다.

3) Causal 마스크

```python
causal_mask = tril(ones(T,T)).view(1,1,T,T)
attn_scores = attn_scores.masked_fill(causal_mask==0, -inf)
```

여기서는 GPT 같은 autoregressive 모델이므로,
현재 토큰은 미래 토큰을 참조하면 안 됩니다.
- 마스크 수식:
$$
M_{ij} = \begin{cases}
1, & i \ge j \
0, & i < j
\end{cases}
$$

즉, i번째 위치는 자기 자신과 과거까지만 볼 수 있고, 미래 j는 차단됩니다.
코드에서는 마스크된 곳에 $-\infty$를 채워 softmax 이후 확률이 0이 되도록 만듭니다.

4) Softmax와 Dropout

```python
attn_probs = F.softmax(attn_scores, dim=-1)
attn_probs = self.attn_dropout(attn_probs)
```

마스크된 attention score에 softmax를 적용하면,
각 Query가 “어떤 Key에 얼마만큼 집중할지” 확률 분포를 얻게 됩니다.
- 수식:
$$
A_h = \text{softmax}(S_h^{\text{masked}})
$$

즉, 한 행(row)은 특정 토큰이 과거 토큰들을 얼마나 주목하는지 나타냅니다.
이때 dropout을 적용하여 일부 attention 연결을 무작위로 제거하면 모델이 과적합을 막을 수 있습니다.

5) 값(Value)의 가중합

```python
attn_out = attn_probs @ v
```

얻은 attention 확률로 Value를 가중합하여 context 벡터를 만듭니다.
- 수식:
$$
Z_h = A_h V_h
$$

직관적으로 말하면, “현재 단어가 필요한 정보”를 과거 단어들의 Value에서 골라 섞어온 결과입니다.

6) 여러 Head 결합

```python
attn_out = attn_out.transpose(1, 2).contiguous().view(B, T, H)
```

각 head에서 얻은 결과 $Z_h$들을 다시 합쳐 $(B, T, H)$ 크기로 만듭니다.
- 수식:
$$
Z = \text{Concat}(Z_1, Z_2, \ldots, Z_{n_h})
$$

즉, 여러 관점(헤드)이 모여 최종적인 맥락(context)을 형성하는 것입니다.

7) 최종 Projection과 Dropout

```python
proj_out = self.c_proj(attn_out)
proj_out = self.proj_dropout(proj_out)
```

Concat된 결과 $Z$는 선형층 $W_O$를 거쳐 다시 모델 hidden dimension으로 투영됩니다.
- 수식:
$$
Y = Z W_O,\quad \tilde{Y} = \text{Dropout}(Y)
$$

이 단계는 “여러 head의 정보를 다시 하나로 잘 섞는 과정”입니다.



In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, hidden_size: int, num_heads: int, dropout_rate: float = 0.1):
        super().__init__()
        assert hidden_size % num_heads == 0, "hidden_size는 num_heads로 나누어떨어져야 합니다."
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        # Q, K, V를 한 번에 생성: (H) -> (3H)
        self.c_attn = nn.Linear(hidden_size, 3 * hidden_size, bias=False)
        # head concat 후 최종 투영
        self.c_proj = nn.Linear(hidden_size, hidden_size, bias=False)

        # 드롭아웃
        self.attn_dropout = nn.Dropout(dropout_rate)   # 어텐션 확률에 적용
        self.proj_dropout = nn.Dropout(dropout_rate)   # 최종 출력에 적용

    def forward(self, x: torch.Tensor, return_attn: bool = False):
        """
        x: (B, T, H)
        return_attn=True이면 어텐션 맵(헤드별)을 반환합니다: (B, num_heads, T, T)
        """
        B, T, H = x.shape
        assert H == self.hidden_size, f"hidden_size 불일치: config={self.hidden_size}, input={H}"

        # 1) Q/K/V 생성
        qkv = self.c_attn(x)                                      # (B, T, 3H)
        q, k, v = qkv.split(self.hidden_size, dim=-1)             # 각 (B, T, H)

        # 2) 멀티헤드로 reshape → (B, T, num_heads, head_dim) → (B, num_heads, T, head_dim)
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2).contiguous() # (B, nh, T, hd)
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1, 2).contiguous() # (B, nh, T, hd)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1, 2).contiguous() # (B, nh, T, hd)

        # 3) 스케일드 닷프로덕트 어텐션
        #    점수: (B, nh, T, T)
        # TODO: attention score를 구현해보세요!
        attn_scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        # 4) Causal Mask (미래 토큰 차단) : (1, 1, T, T) 로 브로드캐스트
        causal_mask = torch.tril(torch.ones(T, T, device=x.device)).view(1, 1, T, T)
        attn_scores = attn_scores.masked_fill(causal_mask == 0, float('-inf'))

        # 5) Softmax → Dropout
        attn_probs = F.softmax(attn_scores, dim=-1)                # (B, nh, T, T)
        attn_probs = self.attn_dropout(attn_probs)

        # 6) 가중합: (B, nh, T, T) @ (B, nh, T, hd) -> (B, nh, T, hd)
        attn_out = attn_probs @ v

        # 7) multi 헤드를 concat: (B, nh, T, hd) -> (B, T, nh*hd=H)
        attn_out = attn_out.transpose(1, 2).contiguous().view(B, T, H)

        # 8) 최종 투영 + 드롭아웃
        proj_out = self.c_proj(attn_out)                           # (B, T, H)
        proj_out = self.proj_dropout(proj_out)

        if return_attn:
            return proj_out, attn_probs
        return proj_out

#### Layer Normalization

LayerNorm은 입력 벡터의 마지막 차원(= hidden dimension) 단위로 평균과 분산을 계산하고, 이를 정규화(normalization)한 뒤 학습 가능한 파라미터(gamma, beta)로 다시 스케일과 시프트를 적용합니다.

즉, 한 시점의 hidden state 전체를 평균 0, 분산 1로 만들고, 이후 적절히 다시 조정하는 과정입니다.

1) 입력과 출력 크기
- 입력: $x \in \mathbb{R}^{B \times T \times H}$ ($B$: batch 크기, $T$: 시퀀스 길이, $H$: hidden dimension)
- 출력: 동일한 크기 $\mathbb{R}^{B \times T \times H}$

2) 평균과 표준편차 계산

각 hidden 벡터($H$차원)에 대해 평균과 표준편차를 구합니다.
- 수식:
$$
\mu = \frac{1}{H} \sum_{i=1}^{H} x_i, \qquad
\sigma = \sqrt{\frac{1}{H}\sum_{i=1}^H (x_i - \mu)^2}
$$

```python
mean = x.mean(dim=-1, keepdim=True) # (B, T, 1)
std  = x.std(dim=-1, keepdim=True)  # (B, T, 1)
```

3) 정규화

평균을 빼고 표준편차로 나누어 hidden 벡터를 정규화합니다.
- 수식:
$$
\hat{x} = \frac{x - \mu}{\sigma + \epsilon}
$$

여기서 $\epsilon$은 분모가 0에 가까워지는 경우를 막기 위한 작은 값입니다.

4) 학습 가능한 스케일($\gamma$)과 시프트($\beta$)

정규화만 하면 표현력이 제한되므로, 학습 가능한 파라미터 $\gamma, \beta \in \mathbb{R}^H$를 도입합니다.
- 최종 출력:
$$
y = \hat{x} \cdot \gamma + \beta
$$


In [20]:

class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5):
        super().__init__()

        self.gamma = nn.Parameter(torch.ones(hidden_size)) # (hidden_size,)
        self.beta = nn.Parameter(torch.zeros(hidden_size)) # (hidden_size,)
        self.eps = eps

    def forward(self, x):
        # x: (batch_size, seq_len, hidden_size)
        mean = x.mean(dim=-1, keepdim=True) # (batch_size, seq_len, 1)
        std = x.std(dim=-1, keepdim=True) # (batch_size, seq_len, 1)
        return (x - mean) / (std + self.eps) * self.gamma + self.beta # (batch_size, seq_len, hidden_size)


# Test layer norm
layer_norm = LayerNorm(config.hidden_size)
x = torch.randn(1, 3, config.hidden_size)
layer_norm_output = layer_norm(x)
print("The size of layer norm output is (batch_size, seq_len, hidden_size):", layer_norm_output.shape)

The size of layer norm output is (batch_size, seq_len, hidden_size): torch.Size([1, 3, 512])


#### MLP

![](https://raw.githubusercontent.com/Ssunbell/TIL/refs/heads/master/assets/MLP.png)

Transformer 블록 안에서 self-attention 다음에 붙는 포지션별(feed-forward) MLP입니다.
즉, 각 토큰(hidden state)에 대해 독립적으로 비선형 변환을 적용합니다.

입력과 출력 크기

- 입력: $x \in \mathbb{R}^{B \times T \times H}$
- 출력: 동일한 크기 $\mathbb{R}^{B \times T \times H}$

여기서 $B$는 배치 크기, $T$는 시퀀스 길이, $H$는 모델 hidden dimension입니다.

내부 구조

MLP는 두 개의 선형 변환과 비선형 활성화 함수로 구성됩니다.
1. 확장 단계 (c_fc)
- 입력 차원 $H$를 더 큰 차원 $H_{ff}$로 확장
- 수식:
$$
z = x W_1 + b_1 \quad \in \mathbb{R}^{B \times T \times H_{ff}}
$$

2. 비선형 활성화 (GELU)
- $z$에 비선형성을 추가
- 수식 (GELU):
$$
\text{GELU}(z) = z \cdot \Phi(z)
$$

여기서 $\Phi(z)$는 Gaussian CDF → ReLU, SwiGLU 등 다른 함수로도 대체 가능

3.	축소 단계 (c_proj)
- 다시 원래 hidden dimension $H$로 줄임
- 수식:
$$
h = z’ W_2 + b_2 \quad \in \mathbb{R}^{B \times T \times H}
$$

4.	Dropout
- 학습 시 일부 뉴런을 무작위로 제거하여 과적합 방지


In [ ]:

class MLP(nn.Module):
    def __init__(self, hidden_size, ff_hidden_size, dropout_rate=0.1):
        super().__init__()

        self.hidden_size = hidden_size
        self.ff_hidden_size = ff_hidden_size

        self.c_fc = nn.Linear(self.hidden_size, self.ff_hidden_size)
        self.act = nn.GELU() # Or other activation functions
        self.c_proj = nn.Linear(self.ff_hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # x: (batch_size, seq_len, hidden_size)
        # TODO: MLP layer를 구현하세요.
        x = self.c_fc(x) # (batch_size, seq_len, ff_hidden_size)
        x = self.act(x) # (batch_size, seq_len, ff_hidden_size)
        x = self.c_proj(x) # (batch_size, seq_len, hidden_size)
        x = self.dropout(x)

        return x

# Test MLP
x = torch.randn(1, 3, config.hidden_size)
mlp = MLP(config.hidden_size, config.ff_hidden_size)
output = mlp(x)
print("Output shape:", output.shape)

Output shape: torch.Size([1, 3, 512])


#### Decoder block

<img src="https://raw.githubusercontent.com/Ssunbell/TIL/refs/heads/master/assets/TransformerBlock.png">

DecoderBlock (GPT-style, Pre-Norm)

- 순서: LayerNorm → Self-Attention → Residual → LayerNorm → MLP → Residual
- Self-Attention은 토큰 간 상호작용을, MLP는 각 토큰의 표현력 강화를 담당
- Pre-Norm 구조는 학습 안정성(gradient 흐름)을 높이는 장점이 큼

입출력과 크기
- 입력/출력: (B, T, H) \to (B, T, H)
- B: 배치, T: 길이, H: hidden size


단계별 동작
1.	LN₁ → Self-Attention → Residual
먼저 입력을 정규화(LN₁)해서 attention에 넣습니다. 이렇게 하면 attention이 “정돈된 스케일”의 벡터를 받아 안정적으로 작동합니다. 계산된 결과를 **원래 입력(residual)**에 더해 정보 손실 없이 업데이트합니다.
2.	LN₂ → MLP → Residual
Self-Attention을 통과해 갱신된 표현을 다시 정규화(LN₂)하고, 포지션별 MLP로 비선형 변환을 가한 뒤 다시 residual add로 합칩니다. 이로써 토큰 간 정보교환(Attention)과 토큰 내 표현강화(MLP)가 한 블록에서 완결됩니다.

간단 수식(개념용)
- $$\text{AttnOut} = \text{MHA}(\text{LN}_1(X))$$
- $$X’ = X + \text{AttnOut}$$
- $$\text{MLPOut} = \text{MLP}(\text{LN}_2(X’))$$
- $$Y = X’ + \text{MLPOut}$$

여기서 MHA는 causal mask를 사용한 multi-head self-attention입니다.

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, hidden_size, ff_hidden_size, num_heads, dropout_rate=0.1):
        super().__init__()
        self.ln_1 = LayerNorm(hidden_size)
        self.attn = MultiHeadSelfAttention(hidden_size, num_heads, dropout_rate)
        self.ln_2 = LayerNorm(hidden_size)
        self.mlp = MLP(hidden_size, ff_hidden_size, dropout_rate)

    def forward(self, x):
        # x: (batch_size, seq_len, hidden_size)
        # TODO: 직접 레이어를 쌓아보세요!
        # Layer norm 1
        x = self.ln_1(x)
        # Self attention + residual
        x = x + self.attn(x)
        # Layer norm 2
        x = self.ln_2(x)
        # MLP + residual
        x = x + self.mlp(x)

        return x
    
# Test Decoder
x = torch.randn(1, 3, config.hidden_size)
decoder = DecoderBlock(config.hidden_size, config.ff_hidden_size, config.num_heads)
x = decoder(x)
print(x.shape)

torch.Size([1, 3, 512])


#### GPT-2

![image](https://raw.githubusercontent.com/Ssunbell/TIL/refs/heads/master/assets/GPT-2.png)

- 입력 토큰 ID → 토큰 임베딩 + 위치 임베딩 → N개의 DecoderBlock(Pre-Norm) → 최종 LayerNorm → 언어 헤드(Linear to vocab) → 로짓(logits)
- Causal self-attention을 통해 미래 토큰을 보지 않도록 학습/추론
- 출력은 (B, T, V) 형태의 다음 토큰 분포의 로짓(softmax 이전 값)

구성요소 설명

1) 임베딩 계층

```python
self.token_embed = nn.Embedding(vocab_size, hidden_size)
self.position_embed = nn.Embedding(max_seq_len, hidden_size)
self.embed_dropout = nn.Dropout(dropout_rate)
```

- 토큰 임베딩: 정수 토큰 ID를 H차원의 연속 벡터로 매핑
- 포지션 임베딩: 각 위치(0..T-1)에 대응하는 H차원 위치 벡터를 더해 순서 정보를 주입
- 드롭아웃: 임베딩 후 과적합 방지

수식(개념):
입력 $X_{\text{ids}} \in {0,\dots,V-1}^{B\times T}$에 대해
$$E = \text{TokenEmb}(X_{\text{ids}}) + \text{PosEmb}([0,\dots,T-1]) \in \mathbb{R}^{B\times T\times H}$$

2) 본문: N개의 DecoderBlock
```python
self.hidden_layers = nn.ModuleList([
    DecoderBlock(hidden_size, ff_hidden_size, num_heads, dropout_rate)
    for _ in range(num_hidden_layers)
])
```

- 각 블록은 Pre-Norm → Self-Attention(+Residual) → Pre-Norm → MLP(+Residual)
- Self-Attention은 causal mask를 사용 (DecoderBlock 내부의 MultiHeadSelfAttention가 수행)

블록별 출력 shape는 입력과 동일: (B, T, H) 유지

3) 최종 정규화 + 언어 헤드
```python
self.ln_f = nn.LayerNorm(hidden_size)
self.language_head = nn.Linear(hidden_size, vocab_size)
```

- 최종 LayerNorm: 깊게 쌓였을 때 분포를 안정화
- 언어 헤드: 각 위치의 hidden state를 어휘 분포 차원으로 사상
- 최종 출력: logits ∈ ℝ^{B×T×V}

개념 수식:
$$H_T = \text{DecoderBlocks}(E) \quad,\quad \tilde{H}_T = \text{LN}_f(H_T) \quad,\quad \text{Logits} = \tilde{H}T W{\text{LM}} + b$$

forward 흐름(말로 설명)
1.	포지션 ID 생성: 0..T-1을 만들어 배치에 broadcast
2.	임베딩 합: 토큰 임베딩 + 위치 임베딩 → 드롭아웃
3.	N개의 디코더 블록 통과: 각 블록에서
    - LN → Causal Self-Attention → Residual
    - LN → MLP → Residual
4.	최종 LN 후 언어 헤드로 투영 → 각 위치의 다음 토큰 로짓


In [ ]:

class GPT2(nn.Module):
    def __init__(
        self,
        hidden_size,
        ff_hidden_size,
        num_heads, 
        vocab_size,
        max_seq_len,
        num_hidden_layers,
        dropout_rate=0.1
    ):
        super().__init__()

        self.token_embed = nn.Embedding(vocab_size, hidden_size)
        self.position_embed = nn.Embedding(max_seq_len, hidden_size)
        self.embed_dropout = nn.Dropout(dropout_rate)

        self.hidden_layers = nn.ModuleList([DecoderBlock(hidden_size, ff_hidden_size, num_heads, dropout_rate) for _ in range(num_hidden_layers)])
        # The final layer norm
        self.ln_f = nn.LayerNorm(hidden_size)

        # The final language head, which maps the last hidden state to logits
        self.language_head = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_ids):
        # input_ids: (batch_size, seq_len)
        batch_size, seq_len = input_ids.shape

        # Create position ids (0, 1, 2, ..., seq_len-1)
        position_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0) # (1, seq_len)

        # Embed tokens and positions, apply dropout
        x = self.token_embed(input_ids) + self.position_embed(position_ids)
        x = self.embed_dropout(x)

        # Pass through decoder blocks
        for layer in self.hidden_layers:
            x = layer(x)

        # Final layer norm
        x = self.ln_f(x)

        # Project to vocabulary
        logits = self.language_head(x) # (batch_size, seq_len, vocab_size)

        return logits


# Test GPT2
input_ids = torch.randint(0, config.vocab_size, (1, 3))
print("Input IDs shape:", input_ids.shape)

model = GPT2(
    hidden_size=config.hidden_size,
    ff_hidden_size=config.ff_hidden_size,
    num_heads=config.num_heads,
    vocab_size=config.vocab_size,
    max_seq_len=config.max_seq_len,
    num_hidden_layers=config.num_hidden_layers,
    dropout_rate=config.dropout_rate,
)
print("Our GPT2 model:\n", model)

logits = model(input_ids)
print("Logits shape:", logits.shape)

Input IDs shape: torch.Size([1, 3])
Our GPT2 model:
 GPT2(
  (token_embed): Embedding(10000, 512)
  (position_embed): Embedding(128, 512)
  (embed_dropout): Dropout(p=0.1, inplace=False)
  (hidden_layers): ModuleList(
    (0-1): 2 x DecoderBlock(
      (ln_1): LayerNorm()
      (attn): MultiHeadSelfAttention(
        (c_attn): Linear(in_features=512, out_features=1536, bias=False)
        (c_proj): Linear(in_features=512, out_features=512, bias=False)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (proj_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm()
      (mlp): MLP(
        (c_fc): Linear(in_features=512, out_features=2048, bias=True)
        (act): GELU(approximate='none')
        (c_proj): Linear(in_features=2048, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (language_head): Linear(in_features=512, out_features=10000, bias=Tru